# Evaluating Models with Scorebook

This notebook demonstrates how to use Scorebook's `evaluate()` function to run inference and compute metrics in a single step.

## When to use `evaluate()`

- You want to run inference on a dataset and score the results
- You're comparing different models on the same dataset
- You want to track hyperparameters alongside results

## Prerequisites

This example uses a local HuggingFace model. For cloud models (OpenAI), see the examples directory.

## Setup

Import necessary modules:

In [ ]:
from pprint import pprint
from typing import Any, List
import transformers

from scorebook import EvalDataset, evaluate

## Initialize Your Model

Set up a HuggingFace pipeline for inference:

In [ ]:
model_name = "microsoft/Phi-4-mini-instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto",
)

print(f"Model loaded: {model_name}")

## Define Your Inference Function

Create a function that processes inputs and returns outputs:

In [ ]:
def inference(inputs: List[Any], **hyperparameters: Any) -> List[Any]:
    """Process inputs through the model.
    
    Args:
        inputs: List of input values from the dataset
        hyperparameters: Model hyperparameters (e.g., temperature, system_message)
        
    Returns:
        List of model outputs
    """
    outputs = []
    
    for input_val in inputs:
        # Build messages for the model
        messages = [
            {
                "role": "system",
                "content": hyperparameters.get("system_message", "You are a helpful assistant.")
            },
            {"role": "user", "content": str(input_val)},
        ]
        
        # Run inference
        result = pipeline(
            messages,
            max_new_tokens=hyperparameters.get("max_new_tokens", 100),
        )
        
        # Extract the answer
        output = str(result[0]["generated_text"][-1]["content"])
        outputs.append(output)
    
    return outputs

## Load Your Dataset

Create an evaluation dataset from a JSON file:

In [ ]:
# Create a sample dataset
sample_data = [
    {"question": "What is 2 + 2?", "answer": "4"},
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "Who wrote Romeo and Juliet?", "answer": "William Shakespeare"},
]

# Create EvalDataset directly from list
dataset = EvalDataset.from_list(
    name="sample_questions",
    metrics="accuracy",
    items=sample_data,
    input="question",
    label="answer",
)

print(f"Loaded dataset with {len(dataset.items)} items")

## Run Evaluation

Use `evaluate()` to run inference and compute metrics:

In [ ]:
results = evaluate(
    inference,
    dataset,
    hyperparameters={
        "system_message": "Answer the question directly and concisely.",
        "max_new_tokens": 50,
    },
    return_aggregates=True,
    return_items=True,
    return_output=True,
    upload_results=False,  # Set to True to upload to Trismik
)

pprint(results)

## Analyze Results

Examine the outputs and metrics:

In [ ]:
# Overall accuracy
print(f"\nOverall Accuracy: {results['aggregate_results'][0]['accuracy']:.2%}")

# Per-item results
print("\nPer-Item Results:")
for i, item in enumerate(results['item_results'], 1):
    print(f"\nQuestion {i}: {item['input']}")
    print(f"  Model Output: {item['output']}")
    print(f"  Expected: {item['label']}")
    print(f"  Correct: {'✓' if item['accuracy'] == 1.0 else '✗'}")

## Hyperparameter Sweeps

Evaluate with different hyperparameters to find optimal settings:

In [ ]:
# Define an inference function
def inference(inputs: List[Any], **hyperparameters: Any) -> List[Any]:
    """Process inputs through the model.

    Args:
        inputs: Input values from an EvalDataset.
        hyperparameters: Model hyperparameters including system_message, temperature, top_p, top_k.

    Returns:
        List of model outputs for all inputs.
    """
    outputs = []
    for input_val in inputs:
        # Preprocess: Build messages
        messages = [
            {"role": "system", "content": hyperparameters["system_message"]},
            {"role": "user", "content": str(input_val)},
        ]

        # Run inference
        result = pipeline(
            messages,
            temperature=hyperparameters["temperature"],
            top_p=hyperparameters.get("top_p"),
            top_k=hyperparameters.get("top_k"),
        )

        # Postprocess: Extract the answer
        output = str(result[0]["generated_text"][-1]["content"])
        outputs.append(output)

    return outputs

# Define hyperparameters with lists of values to create a sweep
hyperparameters = {
    "system_message": "Answer the question directly and concisely.",
    "temperature": [0.6, 0.7, 0.8],
    "top_p": [0.7, 0.8, 0.9],
    "top_k": [10, 20, 30],
}

# Run evaluation across all hyperparameter combinations
results = evaluate(
    inference,
    dataset,
    hyperparameters=hyperparameters,
    return_aggregates=True,
    return_items=True,
    return_output=True,
    upload_results=False,
)

pprint(results)

## Next Steps

- Try the **Adaptive Evaluations** notebook for efficient testing with fewer questions
- See the **Upload Results** notebook to track results in Trismik's dashboard
- Explore batch processing for faster evaluation of large datasets